In [4]:
#!pip install -q transformers torch
!pip install datasets

!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install rouge_score
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 8.6 MB/s 
     |████████████████████████████████| 212 kB 82.9 MB/s 
     |████████████████████████████████| 115 kB 88.7 MB/s 
     |████████████████████████████████| 182 kB 87.7 MB/s 
     |████████████████████████████████| 127 kB 87.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.4 MB/s 
     |████████████████████████████████| 1.3 MB 92.8 MB/s 
     |████████████████████████████████| 7.6 MB 100.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size

In [5]:
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict

from datasets import load_dataset
import pandas as pd
import torch
from transformers.tokenization_utils_base import BatchEncoding
from typing import List, Tuple
from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_metric
import tensorflow as tf
import os
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
from nltk.tokenize import sent_tokenize
import nltk
import evaluate
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
dt3 = load_dataset('csv', data_files = {'train':'/content/gdrive/MyDrive/Copy of cnn_train_class3.csv',
                                       'validation': '/content/gdrive/MyDrive/Copy of cnn_val_class3.csv',
                                       'test': '/content/gdrive/MyDrive/Copy of cnn_test_class3.csv'})

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-de7c395d418ae865/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [8]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model_class3 = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [17]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
!huggingface-cli login

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator_class3 = DataCollatorForSeq2Seq(tokenizer, model=model_class3, return_tensors="tf")

In [11]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["highlights"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_train3 = dt3['train'].map(preprocess_function, batched=True)
tokenized_val3 = dt3['validation'].map(preprocess_function, batched=True)
tokenized_test3 = dt3['test'].map(preprocess_function, batched=True)


  0%|          | 0/65 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [13]:
tokenized_train3=tokenized_train3.remove_columns(dt3['train'].column_names)
tokenized_test3=tokenized_test3.remove_columns(dt3['test'].column_names)
tokenized_val3=tokenized_val3.remove_columns(dt3['validation'].column_names)

features3 = [tokenized_train3[i] for i in range(2)]

In [15]:
tf_train3_dataset = model_class3.prepare_tf_dataset(
    tokenized_train3,
    collate_fn=data_collator_class3,
    shuffle=True,
    batch_size=8)
  
tf_eval3_dataset = model_class3.prepare_tf_dataset(
    tokenized_val3,
    collate_fn=data_collator_class3,
    shuffle=False,
    batch_size=8)
  

num_train_epochs = 8
num_train_steps = len(tf_train3_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01)


model_class3.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [18]:
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-test", tokenizer=tokenizer
)

model_class3.fit(
    tf_train3_dataset, validation_data=tf_eval3_dataset, callbacks=[callback], epochs=8
)

Cloning https://huggingface.co/lyhhhhhh/mt5-small-finetuned-test into local empty directory.


Download file tf_model.h5:   0%|          | 3.47k/2.07G [00:00<?, ?B/s]

Download file tokenizer.json:   0%|          | 8.48k/15.6M [00:00<?, ?B/s]

Download file spiece.model:   0%|          | 1.48k/4.11M [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/4.11M [00:00<?, ?B/s]

...

Clean file tf_model.h5:   0%|          | 1.00k/2.07G [00:00<?, ?B/s]

Epoch 1/8
8014/8014 [==============================] - 926s 113ms/step - loss: 4.0384 - val_loss: 2.3228
Epoch 2/8
8014/8014 [==============================] - 814s 102ms/step - loss: 2.7913 - val_loss: 2.1021
Epoch 3/8
8014/8014 [==============================] - 815s 102ms/step - loss: 2.5264 - val_loss: 1.9837
Epoch 4/8
8014/8014 [==============================] - 817s 102ms/step - loss: 2.4013 - val_loss: 1.9247
Epoch 5/8
8014/8014 [==============================] - 816s 102ms/step - loss: 2.3268 - val_loss: 1.8783
Epoch 6/8
8014/8014 [==============================] - 814s 102ms/step - loss: 2.2781 - val_loss: 1.8712
Epoch 7/8
8014/8014 [==============================] - 812s 101ms/step - loss: 2.2462 - val_loss: 1.8563
Epoch 8/8
8014/8014 [==============================] - 814s 102ms/step - loss: 2.2262 - val_loss: 1.8557


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.33k/2.07G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/lyhhhhhh/mt5-small-finetuned-test
   7d51496..7e28a55  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/lyhhhhhh/mt5-small-finetuned-test
   7d51496..7e28a55  main -> main



In [19]:
from tqdm import tqdm
import numpy as np

generation_data3_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model_class3, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset3 = model_class3.prepare_tf_dataset(
    tokenized_test3,
    collate_fn=generation_data3_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

In [20]:
@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model_class3.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )

In [24]:
all_preds3_class3 = []
all_labels_class3 = []
for batch, labels in tqdm(tf_generate_dataset3):
    predictions = generate_with_xla(batch)
    decoded_preds3 = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels3 = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds3 = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds3]
    decoded_labels3 = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels3]
    all_preds3_class3.extend(decoded_preds3)
    all_labels_class3.extend(decoded_labels3)

100%|██████████| 291/291 [00:25<00:00, 11.31it/s]


In [25]:
rouge_score = evaluate.load("rouge")
result_class3 = rouge_score.compute(
    predictions=decoded_preds3, references=decoded_labels3, use_stemmer=True
)
result_class3 = {key: value* 100 for key, value in result_class3.items()}
{k: round(v, 4) for k, v in result_class3.items()}

{'rouge1': 47.1936, 'rouge2': 30.0524, 'rougeL': 41.9029, 'rougeLsum': 44.4177}

In [29]:
import pandas as pd
df_class3_test = pd.DataFrame(all_preds3_class3)
len(df_class3_test)

2328

In [32]:
df_test3 = pd.DataFrame(dt3['test'])
df_test3_new = df_test3.iloc[:2328, 1:]
df_test3_new.head()

,article,highlights,class
0,(CNN)Paul Walker is hardly the first actor to ...,"""Furious 7"" pays tribute to star Paul Walker, ...",3
1,"(CNN)The Rev. Robert H. Schuller, California t...","The Rev. Robert Schuller, 88, had been diagnos...",3
2,(CNN)Just as mimeograph machines and photocopi...,Going online has become the path of least resi...,3
3,(CNN)Standing up for what you believe. What do...,Indiana town's Memories Pizza is shut down aft...,3
4,(CNN)Sky watchers in western North America are...,The total eclipse will only last 4 minutes and...,3


In [33]:
df_test3_new['summary'] = df_class3_test
df_test3_new.head()

,article,highlights,class,summary
0,(CNN)Paul Walker is hardly the first actor to ...,"""Furious 7"" pays tribute to star Paul Walker, ...",3,"""Furious 7"" is the first actor to die during a..."
1,"(CNN)The Rev. Robert H. Schuller, California t...","The Rev. Robert Schuller, 88, had been diagnos...",3,"Rev.\nRobert H. Schuller, California, was diag..."
2,(CNN)Just as mimeograph machines and photocopi...,Going online has become the path of least resi...,3,"Amos Yee, the teenage video blogger, was arres..."
3,(CNN)Standing up for what you believe. What do...,Indiana town's Memories Pizza is shut down aft...,3,Memories Pizza in Indiana town of Walkerton is...
4,(CNN)Sky watchers in western North America are...,The total eclipse will only last 4 minutes and...,3,The total lunar eclipse started at 3:16 a.m. P...


In [34]:
df_test3_new.to_csv('test_class3_final.csv')

In [35]:
!cp test_class3_final.csv "gdrive/My Drive"

In [36]:
model_class3.push_to_hub("lyhhhhhh/mt5-small-finetuned-test-class3")